https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/centerpartiets-budgetmotion-2022_H9024121

In [3]:
import pandas as pd

In [1]:
html_url = 'https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/centerpartiets-budgetmotion-2022_H9024121/html'

In [4]:
tables = pd.read_html(html_url)

In [6]:
tables[0]

,0,1
0,Annie Lööf (C),NaN
1,Anders W Jonsson (C),Martin Ådahl (C)
2,Alireza Akhondi (C),Malin Björk (C)
3,Daniel Bäckström (C),Jonny Cato (C)
4,Fredrik Christensson (C),Catarina Deremar (C)
5,Magnus Ek (C),Johan Hedin (C)
6,Ulrika Heie (C),Peter Helander (C)
7,Martina Johansson (C),Ola Johansson (C)
8,Mikael Larsson (C),Helena Lindahl (C)
9,Kerstin Lundgren (C),Aphram Melki (C)


In [12]:
tables = tables[20:47]

In [16]:
for table in tables:
    table.columns = ['Kod', 'Namn', 'Regeringens förslag', 'Avvikelse från regeringen']
    table.drop([0], inplace=True)

In [17]:
df = pd.concat(tables, sort=False)

In [18]:
df = df.dropna(how='all')

In [19]:
df = df[df.Kod != 'Summa']

In [20]:
df = df[~df.Kod.str.startswith('Ramanslag')]

In [21]:
df = df.replace('±0', '0')

In [23]:
df.to_csv('sverigedemokraterna-budgetmotion-2022.csv', index=False)